In [2]:
# Import the libraries
import numpy as np
import pandas as pd
from skimage import measure
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

In [3]:
# Read the GT RLE and meta data ***** Colon Data *****
ground_truth_rle = pd.read_csv(r'C:\Users\soodn\Downloads\Naveksha\Kaggle HuBMAP\Data\colon-data-reprocessed\test.csv')
df_info = pd.read_csv(r'C:\Users\soodn\Downloads\Naveksha\Kaggle HuBMAP\Data\colon-data-reprocessed\HuBMAP-20-dataset_information.csv')

In [4]:
# Read the GT RLE and meta data ***** Kidney Data *****
ground_truth_rle = pd.read_csv(r'C:\Users\soodn\Downloads\Naveksha\Kaggle HuBMAP\Data\kidney-data\test.csv')
df_info = pd.read_csv(r'C:\Users\soodn\Downloads\Naveksha\Kaggle HuBMAP\Data\kidney-data\HuBMAP-20-dataset_information.csv')

In [5]:
# Read PREDICTED RLE
predicted_rle = pd.read_csv(r'C:\Users\soodn\Downloads\Naveksha\Kaggle HuBMAP\Scripts\4. DeepLive\submission-deeplive.csv')

In [6]:
# Utility function to convert rle to mask
def enc2mask(encs, shape):
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for m, enc in enumerate(encs):
        print (m, encs)
        if isinstance(enc, np.float) and np.isnan(enc):
            continue
        enc_split = enc.split()
        for i in range(len(enc_split) // 2):
            start = int(enc_split[2 * i]) - 1
            length = int(enc_split[2 * i + 1])
            img[start: start + length] = 1 + m

    return img.reshape(shape).T

In [8]:
# Function to make a 3D list of mask
def polygon_outline_list (gt_mask, pred_mask):
    contours_gt = measure.find_contours(gt_mask, 0.8)
    polygons_gt = []
    for object in contours_gt:
        coords = []
        for point in object:
            coords.append([int(point[0]), int(point[1])])
        polygons_gt.append(coords)

    contours_pred = measure.find_contours(pred_mask, 0.8)
    polygons_pred = []
    for object in contours_pred:
        coords = []
        for point in object:
            coords.append([int(point[0]), int(point[1])])
        polygons_pred.append(coords)
    return polygons_gt, polygons_pred

In [12]:
# Function to count the missed gloms,FNs and FPs
def find_missed(polygons_l1, polygons_l2, matching_threshold):
    mt = matching_threshold
    ## Using shapely
    count = 0
    for poly_1 in polygons_l1:
        for poly_2 in polygons_l2:
            p = Polygon(poly_1)
            q = Polygon(poly_2)
            if (p.intersection(q).area / p.area >=matching_threshold):
                count += 1
    return count

In [ ]:
# Find the False positives and false negatives in all the test files 
for idx, row in ground_truth_rle.iterrows():
    IMAGE_ID = ground_truth_rle['id'][idx]

    shape = (df_info['width_pixels'][idx], df_info['height_pixels'][idx])

    gt_rle = ground_truth_rle[ground_truth_rle['id'] == IMAGE_ID]["encoding"]
    gt_mask = enc2mask(gt_rle, shape)
    plt.imshow(gt_mask)
    plt.show()

    pred_rle = predicted_rle[predicted_rle['id'] == IMAGE_ID]["predicted"]
    pred_mask = enc2mask(pred_rle, shape)
    plt.imshow(pred_mask)
    plt.show()


    polygons_gt, polygons_pred = polygon_outline_list (gt_mask, pred_mask)
    matching_threshold = 0.8

    false_negatives = len(polygons_gt) -  find_missed(polygons_gt, polygons_pred, matching_threshold)
    false_positives = len(polygons_pred) - find_missed(polygons_pred, polygons_gt, matching_threshold)

    print (f"For matching threshold = {matching_threshold},")
    print (f"Of {len(polygons_gt)} number of gloms/cypts in ground truth, {false_negatives} are missed in prediction.")
    print (f"False Negative Rate = {false_negatives/len(polygons_gt)}")
    print (f"Of {len(polygons_pred)} number of gloms/crypts in prediction,  {false_positives} aren't there in ground truth.")
    print (f"False Positive Rate = {false_positives/len(polygons_pred)}")
    print ("================================================================================\n")

0 0    60954223 34 60978180 69 61002168 74 61026156 7...
Name: encoding, dtype: object


<ipython-input-6-ed672ee9e710>:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(enc, np.float) and np.isnan(enc):


In [ ]:
# # match_threshold = 0.8
# for poly_gt in polygons_gt: 
#     for poly_pred in polygons_pred:
#         i+=1
#         pixel_count = 0
#         for coord_gt in poly_gt:
#             for coord_pred in poly_pred:
#                 if coord_gt == coord_pred:
#                     pixel_count += 1
#         print (j, i, pixel_count, len(poly_gt), len(poly_pred))
#         if pixel_count > 10:
#             count += 1
#             print ("*************************",j,i,"Matched")